In [153]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests


In [154]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)



In [155]:
#run above function 'init_browser'
browser = init_browser()



In [156]:
# Scrape the NASA Mars News Site and collect the latest News
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

time.sleep(1)

In [157]:
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [158]:
# Get the News Info Section
news_info = soup.find('div', class_='list_text')
print(news_info.text)

August 24, 2020NASA Engineers Checking InSight's Weather SensorsAn electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.


In [159]:
# collect the latest News Title
news_sec1 = news_info.find('div', class_='content_title')
news_title = news_sec1.a.text
print(news_title)

NASA Engineers Checking InSight's Weather Sensors


In [160]:
# news article Paragraph Text
news_p = news_info.find('div', class_="article_teaser_body").text
print(news_p)
# Close the browser after scraping
browser.quit()

An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.


In [187]:
### LETS GET THE JPL IMAGE!!
# Visit the url for JPL Featured Space Image here

#run above function 'init_browser'
browser = init_browser()

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

time.sleep(10)

In [188]:
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")


In [191]:
# Use splinter to navigate the site and find the 
image_button = browser.find_by_id('full_image')

image_button.click()


TypeError: 'NoneType' object is not callable

None


In [128]:
#image url for the current Featured Mars Image and 
#assign the url string to a variable called featured_image_url
image_button = browser.find_by_id('')
image_page = image_button.click()

AttributeError: 'NoneType' object has no attribute 'find'

In [93]:
featured_image_url = soup.find('img')[0]["src"]
featured_image_url

KeyError: 0

In [ ]:
featured_image_url = url + relative_image_path

# Store data in a dictionary
mars_data = {
    "news_title": news_title,
    "news_p": news_p,
    "featured_image_url": featured_image_url
}

# Close the browser after scraping
browser.quit()